### Load Libraries 

In [19]:
import tensorflow as tf
from keras_vggface.vggface import VGGFace

In [20]:
from keras.models import Model, Sequential

In [21]:
from keras.layers import Dense, Dropout, Lambda, Flatten
import csv
import cv2
import numpy as np
from scipy import misc
from keras_vggface import utils
from keras.preprocessing import image
from sklearn import preprocessing
from scipy.spatial import distance
import os
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity

### Select Architecture

In [14]:
def l2_norm(x):
    #x2=tf.nn.l2_normalize(x,axis=None,epsilon=1e-12,name=None,dim=None)
    x2=tf.nn.l2_normalize(x,dim=1)
    return x2


# CHOOSE AN ARCHITECTURE:  
#architecture = 'vgg'
architecture = 'resnet'


dirpath = os.getcwd()

# The images that we are using are already cropped, 
# so we don't need to detect face positions

bbox_1 = [0,0,224,224]
bbox_2 = [0,0,224,224]


if architecture == 'vgg':
    #baseline model with vgg:
    vgg_model = VGGFace(model = 'vgg16')
#                        CHANGE LAYER HERE  \/
    feature_layer = vgg_model.get_layer('fc7/relu').output
    model = Model(vgg_model.input, feature_layer)
    version = 1 


elif architecture == 'resnet':
    #baseline model with resnet:
    resnet = VGGFace(model = 'resnet50')
    
#                   CHANGE LAYER HERE  \/
    last_layer = resnet.get_layer('conv5_3_3x3').output
#     last_layer = resnet.get_layer('conv2_1_1x1_proj').output

    feature_layer = Flatten(name='flatten')(last_layer)
    model = Model(resnet.input, feature_layer)
    version = 2   


### Create method to get distance between 2 images:
Note: This code can return Cosine Similarity (1=Match) and Euclidean Distance (0=Match)

In [20]:

def RunModel(img_url_1,img_url_2):
    # first image:
#     img_url_1 = dirpath + img_url_1
#     img_url_2 = dirpath + img_url_2
    img_1 = cv2.imread(img_url_1)
#     img_1 = img_1/255
#     img_1 = img_1[bbox_1[1]:bbox_1[1]+bbox_1[3],bbox_1[0]:bbox_1[0]+bbox_1[2],:]
#     img_1 = misc.imresize(img_1, (224, 224), interp='bilinear')
    img_1 = image.img_to_array(img_1)
    img_1 = np.expand_dims(img_1, axis=0)
    img_1 = utils.preprocess_input(img_1, version=version) # version=1 for vgg16, version=2 for resnet50
    predict_1 = model.predict(img_1) # predict with baseline model
    baseline_features_1 = preprocessing.normalize(predict_1, norm='l2', axis=1, copy=True, return_norm=False)

    # second image:
    img_2 = cv2.imread(img_url_2)
#     img_2 = img_2/255
#     img_2 = img_2[bbox_2[1]:bbox_2[1]+bbox_2[3],bbox_2[0]:bbox_2[0]+bbox_2[2],:]
#     img_2 = misc.imresize(img_2, (224, 224), interp='bilinear')
    img_2 = image.img_to_array(img_2)
    img_2 = np.expand_dims(img_2, axis=0)
    img_2 = utils.preprocess_input(img_2, version=version) # version=1 for vgg16, version=2 for resnet50
    predict_2 = model.predict(img_2) # predict with baseline model
    baseline_features_2 = preprocessing.normalize(predict_2, norm='l2', axis=1, copy=True, return_norm=False)


    # Calculate distances:
    baseline_distance = distance.euclidean(baseline_features_1, baseline_features_2)
    cosine_distance = cosine_similarity(baseline_features_1, baseline_features_2)
    
    return(cosine_distance,baseline_distance)

In [15]:
def GetFeatureVector(img_url_1):
        # first image:
#     img_url_1 = dirpath + img_url_1
#     img_url_2 = dirpath + img_url_2
    img_1 = cv2.imread(img_url_1)
#     img_1 = img_1/255
#     img_1 = img_1[bbox_1[1]:bbox_1[1]+bbox_1[3],bbox_1[0]:bbox_1[0]+bbox_1[2],:]
#     img_1 = misc.imresize(img_1, (224, 224), interp='bilinear')
    img_1 = image.img_to_array(img_1)
    img_1 = np.expand_dims(img_1, axis=0)
    img_1 = utils.preprocess_input(img_1, version=version) # version=1 for vgg16, version=2 for resnet50
    predict_1 = model.predict(img_1) # predict with baseline model
    baseline_features_1 = preprocessing.normalize(predict_1, norm='l2', axis=1, copy=True, return_norm=False)
    return (baseline_features_1)

### Run on images

In [16]:
list_dir = os.listdir('Database/')

In [17]:
list_dir = ['OBAMA.png']

In [18]:
np.shape(GetFeatureVector('OBAMA.png'))

(1, 25088)

In [16]:
# Run the model on all images of a certain folder:

model_result =[] # Matchs Scores
for i in range(len(list_dir)):
    print(i)
    for j in range(len(list_dir)):
        model_result.append(RunModel('Database/'+list_dir[i],
                                 'Database/'+list_dir[j])[1]) #set 1 for Euclidean Distance, 0 for cosine similarity

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149


In [17]:
# Show the scores
for i in range(len(model_result)):
    print(model_result[i])

0.0
0.8144626617431641
1.1971707344055176
1.2156782150268555
1.188852310180664
1.2651509046554565
1.249467134475708
1.2095468044281006
1.1650159358978271
1.2379624843597412
1.2394905090332031
1.2169160842895508
1.2569537162780762
1.2338985204696655
1.2112839221954346
1.202566385269165
1.233049988746643
1.2586826086044312
1.2610125541687012
1.2637813091278076
1.1382073163986206
1.0835998058319092
1.2427226305007935
1.2085751295089722
1.1869066953659058
1.2016922235488892
1.2225935459136963
1.229736328125
1.2231107950210571
1.1781654357910156
1.2578465938568115
1.2468522787094116
1.200076937675476
1.2926177978515625
1.2685236930847168
1.1196699142456055
1.2254492044448853
1.267542839050293
1.2733924388885498
1.261358380317688
1.2327808141708374
1.2427699565887451
1.1481292247772217
1.1431307792663574
1.1053192615509033
1.2286021709442139
1.2553629875183105
1.2716127634048462
1.2789160013198853
1.2423596382141113
1.2520473003387451
1.2657711505889893
1.2505128383636475
1.2746305465698242


In [18]:
# Reshape to become a square matrix:
model_result = np.reshape(model_result,(len(list_dir),-1))

In [19]:
# Show square matrix:

model_result

array([[0.        , 0.81446266, 1.19717073, ..., 1.26810563, 1.22160876,
        1.25843763],
       [0.81446266, 0.        , 1.16269171, ..., 1.24958158, 1.20264161,
        1.22685301],
       [1.19717073, 1.16269171, 0.        , ..., 1.2811054 , 1.28657353,
        1.26991796],
       ...,
       [1.26810563, 1.24958158, 1.2811054 , ..., 0.        , 0.80509824,
        0.75435418],
       [1.22160876, 1.20264161, 1.28657353, ..., 0.80509824, 0.        ,
        0.64657342],
       [1.25843763, 1.22685301, 1.26991796, ..., 0.75435418, 0.64657342,
        0.        ]])

In [20]:
# save matrix
np.savetxt('Euclidean_Distances.csv',model_result,
          delimiter=',')

In [23]:
with open('filenames.csv','a+') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    wr.writerow(list_dir)